In [1]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.python.client import device_lib
from tensorflow.keras.models import Model

# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True,threshold=np.inf)
pd.options.display.max_rows = 999

2023-02-20 14:14:28.088897: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-20 14:14:28.204485: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: 
2023-02-20 14:14:28.204499: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-20 14:14:28.770442: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No s

In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

df = pd.read_csv("kddcup99_csv.csv")
col2encode = ["protocol_type","service","flag"]
ohe_col = pd.get_dummies(df[col2encode],columns=col2encode)
df = df.drop(columns=col2encode)
df_ohe = pd.concat([df,ohe_col],axis=1)

attack_values = pd.unique(df_ohe["label"].values.ravel())
attack_values = attack_values[1:] ## Remove "normal" as it is not an attack

### Create a dict to remap attack to True and normal to False
d_attack = {"normal":False}
for a in attack_values:
    d_attack.update({a:True})

label = df_ohe["label"].map(d_attack)
df_ohe = df_ohe.drop(columns="label",axis=1)

X_train, X_test, y_train, y_test = train_test_split(df_ohe, label, test_size=0.25, random_state=42)

scaler = StandardScaler()
scaler.fit(X_train)
X_sc_train = scaler.transform(X_train)
X_sc_test = scaler.transform(X_test)

NCOMPONENTS = 36

pca = PCA(n_components=NCOMPONENTS)
X_pca_train = pca.fit_transform(X_sc_train)
X_pca_test = pca.transform(X_sc_test)

print(X_pca_train.shape)
print(X_pca_test.shape)

X_pca_train  = X_pca_train.reshape(370515,6,6)
X_pca_test  = X_pca_test.reshape(123505,6,6)

print(X_pca_train.shape)
print(X_pca_test.shape)


(370515, 36)
(123505, 36)
(370515, 6, 6)
(123505, 6, 6)


In [3]:

from keras import Input
from keras.regularizers import l1
from keras.layers import MaxPooling2D, Flatten, BatchNormalization, Dense
from qkeras import QConv2DBatchnorm, QActivation, QDense

x = x_in = Input(shape=(6,6,1))

x = QConv2DBatchnorm(32, kernel_size=(3,3), strides=(1,1),
                     kernel_quantizer="quantized_bits(6,0,alpha=1)",
                     bias_quantizer="quantized_bits(6,0,alpha=1)",
                     kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001), use_bias=True,
                     name='fused_convbn')(x)
x = QActivation('quantized_relu(6)',name='conv_act1')(x)
x = MaxPooling2D(pool_size = (2,2),name='pool')(x)
x = Flatten()(x)
x = BatchNormalization(name='bn_dense')(x)
x_out = Dense(1,
           name='output_dense')(x)
#x_out = Activation('softmax',name='output_softmax')(x)
model = Model(inputs=[x_in], outputs=[x_out], name='qkeras')

model.summary()

2023-02-20 14:14:37.800850: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: 
2023-02-20 14:14:37.800896: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: 
2023-02-20 14:14:37.800922: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: 
2023-02-20 14:14:37.800947: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
Model: "qkeras"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 6, 6, 1)]         0         
                                                                 
 fused_convbn (QConv2DBatchn  (None, 4, 4, 32)         449       
 orm)                                                            
                                                                 
 conv_act1 (QActivation)     (None, 4, 4, 32)          0         
                                                                 
 pool (MaxPooling2D)         (None, 2, 2, 32)          0         
                                                                 
 flatten (Flatten)           (None, 128)               0 

In [4]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss=keras.losses.BinaryCrossentropy(),
    metrics =['accuracy']
)

model.fit(X_pca_train, y_train, batch_size=64, epochs=10,validation_data=(X_pca_test,y_test))

Epoch 1/10
5790/5790 [==============================] - 42s 7ms/step - loss: 0.5786 - accuracy: 0.9606 - val_loss: 0.4297 - val_accuracy: 0.9723
Epoch 2/10
5790/5790 [==============================] - 41s 7ms/step - loss: 0.2393 - accuracy: 0.9835 - val_loss: 0.1640 - val_accuracy: 0.9894
Epoch 3/10
5790/5790 [==============================] - 41s 7ms/step - loss: 0.1875 - accuracy: 0.9864 - val_loss: 0.1207 - val_accuracy: 0.9896
Epoch 4/10
5790/5790 [==============================] - 41s 7ms/step - loss: 0.1306 - accuracy: 0.9904 - val_loss: 0.2123 - val_accuracy: 0.9857
Epoch 5/10
5790/5790 [==============================] - 41s 7ms/step - loss: 0.1568 - accuracy: 0.9889 - val_loss: 0.1348 - val_accuracy: 0.9913
Epoch 6/10
5790/5790 [==============================] - 40s 7ms/step - loss: 0.1649 - accuracy: 0.9889 - val_loss: 0.2636 - val_accuracy: 0.9831
Epoch 7/10
5790/5790 [==============================] - 40s 7ms/step - loss: 0.1199 - accuracy: 0.9916 - val_loss: 0.0872 - val_ac

In [5]:
print("Evaluate on test data")
results = model.evaluate(X_pca_test, y_test)
print("test acc:", results)
#model.save("model3")


Evaluate on test data
3860/3860 [==============================] - 10s 3ms/step - loss: 0.0518 - accuracy: 0.9965
test acc: [0.05184793472290039, 0.996453583240509]


In [6]:
from tensorflow_model_optimization.sparsity.keras import strip_pruning
from tensorflow_model_optimization.python.core.sparsity.keras import pruning_wrapper
from qkeras.utils import _add_supported_quantized_objects

co = {}
_add_supported_quantized_objects(co)
co['PruneLowMagnitude'] = pruning_wrapper.PruneLowMagnitude

qmodel  = strip_pruning(model)

In [7]:
import hls4ml
import plotting
import os

os.environ['PATH'] = '/opt/Xilinx/Vivado/2019.2/bin:' + os.environ['PATH']

# Then the QKeras model
hls4ml.model.optimizer.OutputRoundingSaturationMode.layers = ['Activation']
hls4ml.model.optimizer.OutputRoundingSaturationMode.rounding_mode = 'AP_RND'
hls4ml.model.optimizer.OutputRoundingSaturationMode.saturation_mode = 'AP_SAT'

hls_config_q = hls4ml.utils.config_from_keras_model(qmodel, granularity='name')
hls_config_q['Model']['ReuseFactor'] = 1
hls_config_q['Model']['Precision'] = 'ap_fixed<6,0>'
  
cfg_q = hls4ml.converters.create_config(backend='Vivado')
cfg_q['IOType']     = 'io_stream' # Must set this if using CNNs!
cfg_q['HLSConfig']  = hls_config_q
cfg_q['KerasModel'] = qmodel
cfg_q['OutputDir']  = 'quantized_pruned_cnn/'
cfg_q['XilinxPart'] = 'xc7z020clg484-1'
  
hls_model_q = hls4ml.converters.keras_to_hls(cfg_q)
hls_model_q.compile()

Interpreting Model
Topology:
Layer name: input_1, layer type: Input
Layer name: fused_convbn, layer type: QConv2DBatchnorm
Layer name: conv_act1, layer type: QActivation
Layer name: pool, layer type: MaxPooling2D
Layer name: bn_dense, layer type: BatchNormalization
Layer name: output_dense, layer type: Dense
  -> Activation (linear), layer name: output_dense
Interpreting Model
Topology:
Layer name: input_1, layer type: InputLayer, input shapes: [[None, 6, 6, 1]], output shape: [None, 6, 6, 1]
Layer name: fused_convbn, layer type: QConv2DBatchnorm, input shapes: [[None, 6, 6, 1]], output shape: [None, 4, 4, 32]
Layer name: conv_act1, layer type: Activation, input shapes: [[None, 4, 4, 32]], output shape: [None, 4, 4, 32]
Layer name: pool, layer type: MaxPooling2D, input shapes: [[None, 4, 4, 32]], output shape: [None, 2, 2, 32]
Layer name: flatten, layer type: Reshape, input shapes: [[None, 2, 2, 32]], output shape: [None, 128]
Layer name: bn_dense, layer type: BatchNormalization, input

/home/bertelem/Documents/env_python3/lib/python3.7/site-packages/hls4ml/converters/__init__.py:16: UserWarning: WARNING: Pytorch converter is not enabled!
  warnings.warn("WARNING: Pytorch converter is not enabled!")


Done


In [8]:
hls_model_q.build(csim=False, synth=True, vsynth=True)


****** Vivado(TM) HLS - High-Level Synthesis from C, C++ and SystemC v2019.2 (64-bit)
  **** SW Build 2708876 on Wed Nov  6 21:39:14 MST 2019
  **** IP Build 2700528 on Thu Nov  7 00:09:20 MST 2019
    ** Copyright 1986-2019 Xilinx, Inc. All Rights Reserved.

source /opt/Xilinx/Vivado/2019.2/scripts/vivado_hls/hls.tcl -notrace
INFO: [HLS 200-10] Running '/opt/Xilinx/Vivado/2019.2/bin/unwrapped/lnx64.o/vivado_hls'
INFO: [HLS 200-10] For user 'bertelem' on host 'tdell5823.ensieta.ecole' (Linux_x86_64 version 4.19.0-22-amd64) on Mon Feb 20 14:21:39 CET 2023
INFO: [HLS 200-10] On os Debian GNU/Linux 10 (buster)
INFO: [HLS 200-10] In directory '/home/bertelem/Documents/CNNKDD/quantized_pruned_cnn'
Sourcing Tcl script 'build_prj.tcl'
INFO: [HLS 200-10] Creating and opening project '/home/bertelem/Documents/CNNKDD/quantized_pruned_cnn/myproject_prj'.
INFO: [HLS 200-10] Adding design file 'firmware/myproject.cpp' to the project
INFO: [HLS 200-10] Adding test bench file 'myproject_test.cpp' to

INFO: [XFORM 203-602] Inlining function 'fp_struct<double>::mantissa' into 'generic_cast_IEEE754<int, (ap_q_mode)6, double>' (/wrk/2019.2/continuous/2019_11_06_2708876/src/products/hls/hls_lib/hlsmath/include/FloatingPoint/hls_case_IEEE754.h:15) automatically.
INFO: [XFORM 203-602] Inlining function 'fp_struct<double>::expv' into 'generic_cast_IEEE754<int, (ap_q_mode)6, double>' (/wrk/2019.2/continuous/2019_11_06_2708876/src/products/hls/hls_lib/hlsmath/include/FloatingPoint/hls_case_IEEE754.h:18) automatically.
INFO: [XFORM 203-602] Inlining function 'fp_struct<double>::__signbit' into 'generic_cast_IEEE754<int, (ap_q_mode)6, double>' (/wrk/2019.2/continuous/2019_11_06_2708876/src/products/hls/hls_lib/hlsmath/include/FloatingPoint/hls_case_IEEE754.h:59) automatically.
INFO: [XFORM 203-602] Inlining function 'generic_cast_IEEE754<int, (ap_q_mode)6, double>' into 'generic_cast_IEEE754<int, double>' (/wrk/2019.2/continuous/2019_11_06_2708876/src/products/hls/hls_lib/hlsmath/include/Float

INFO: [HLS 200-489] Unrolling loop 'LinearPackLoop' (firmware/nnet_utils/nnet_activation_stream.h:45) in function 'nnet::linear<nnet::array<ap_fixed<6, 0, (ap_q_mode)5, (ap_o_mode)3, 0>, 32u>, nnet::array<ap_fixed<6, 0, (ap_q_mode)0, (ap_o_mode)0, 0>, 32u>, linear_config3>' completely with a factor of 32.
INFO: [HLS 200-489] Unrolling loop 'DataPack' (firmware/nnet_utils/nnet_dense_stream.h:46) in function 'nnet::dense<nnet::array<ap_fixed<6, 0, (ap_q_mode)5, (ap_o_mode)3, 0>, 128u>, nnet::array<ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, 1u>, config8>' completely with a factor of 128.
INFO: [HLS 200-489] Unrolling loop 'Product1' (firmware/nnet_utils/nnet_dense_latency.h:85) in function 'nnet::conv_2d_cl<nnet::array<ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, 1u>, nnet::array<ap_fixed<6, 0, (ap_q_mode)5, (ap_o_mode)3, 0>, 32u>, config2>' completely with a factor of 9.
INFO: [HLS 200-489] Unrolling loop 'Product2' (firmware/nnet_utils/nnet_dense_latency.h:90) in function 'nnet:

INFO: [XFORM 203-101] Partitioning array 'res_out.i.i'  in dimension 1 completely.
INFO: [XFORM 203-101] Partitioning array 'b2.V'  in dimension 1 completely.
INFO: [XFORM 203-101] Partitioning array 'mult.V' (firmware/nnet_utils/nnet_dense_latency.h:39) in dimension 1 completely.
INFO: [XFORM 203-101] Partitioning array 'acc.V' (firmware/nnet_utils/nnet_dense_latency.h:40) in dimension 1 completely.
INFO: [XFORM 203-101] Partitioning array 'b8.V'  in dimension 1 completely.
INFO: [XFORM 203-101] Partitioning array 'mult.V' (firmware/nnet_utils/nnet_dense_latency.h:39) in dimension 1 completely.
INFO: [XFORM 203-101] Partitioning array 'acc.V' (firmware/nnet_utils/nnet_dense_latency.h:40) in dimension 1 completely.
INFO: [XFORM 203-101] Partitioning array 'shift_buffer.V' (firmware/nnet_utils/nnet_conv_stream.h:217) in dimension 1 completely.
INFO: [XFORM 203-101] Partitioning array 'shift_buffer.V' (firmware/nnet_utils/nnet_conv_stream.h:217) in dimension 1 completely.
INFO: [XFORM 20

INFO: [HLS 200-111] Finished Pre-synthesis Time (s): cpu = 00:00:36 ; elapsed = 00:00:37 . Memory (MB): peak = 1100.719 ; gain = 686.012 ; free physical = 25178 ; free virtual = 27862
INFO: [XFORM 203-541] Flattening a loop nest 'ReadInputHeight' (firmware/nnet_utils/nnet_pooling_stream.h:233:80) in function 'nnet::pooling2d_cl<nnet::array<ap_ufixed<6, 0, (ap_q_mode)0, (ap_o_mode)0, 0>, 32u>, nnet::array<ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, 32u>, config5>'.
INFO: [XFORM 203-541] Flattening a loop nest 'ReadInputHeight' (firmware/nnet_utils/nnet_conv2d_stream.h:78:80) in function 'nnet::conv_2d_cl<nnet::array<ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, 1u>, nnet::array<ap_fixed<6, 0, (ap_q_mode)5, (ap_o_mode)3, 0>, 32u>, config2>'.
INFO: [HLS 200-111] Finished Architecture Synthesis Time (s): cpu = 00:00:44 ; elapsed = 00:00:45 . Memory (MB): peak = 1276.715 ; gain = 862.008 ; free physical = 25073 ; free virtual = 27761
INFO: [HLS 200-10] Starting hardware synthesis ...


...

INFO: [Synth 8-3886] merging instance 'i_1_0/layer7_out_V_data_34_V_U/U_fifo_w6_d1_A_shiftReg/SRL_SIG_reg[0][0]' (FDE) to 'i_1_0/layer7_out_V_data_34_V_U/U_fifo_w6_d1_A_shiftReg/SRL_SIG_reg[0][1]'
INFO: [Synth 8-3886] merging instance 'i_1_0/layer7_out_V_data_34_V_U/U_fifo_w6_d1_A_shiftReg/SRL_SIG_reg[0][1]' (FDE) to 'i_1_0/layer7_out_V_data_34_V_U/U_fifo_w6_d1_A_shiftReg/SRL_SIG_reg[0][2]'
INFO: [Synth 8-3886] merging instance 'i_1_0/layer7_out_V_data_34_V_U/U_fifo_w6_d1_A_shiftReg/SRL_SIG_reg[0][2]' (FDE) to 'i_1_0/layer7_out_V_data_34_V_U/U_fifo_w6_d1_A_shiftReg/SRL_SIG_reg[0][3]'
INFO: [Synth 8-3886] merging instance 'i_1_0/layer7_out_V_data_34_V_U/U_fifo_w6_d1_A_shiftReg/SRL_SIG_reg[0][3]' (FDE) to 'i_1_0/layer7_out_V_data_34_V_U/U_fifo_w6_d1_A_shiftReg/SRL_SIG_reg[0][4]'
INFO: [Synth 8-3886] merging instance 'i_1_0/layer7_out_V_data_34_V_U/U_fifo_w6_d1_A_shiftReg/SRL_SIG_reg[0][4]' (FDE) to 'i_1_0/layer7_out_V_data_34_V_U/U_fifo_w6_d1_A_shiftReg/SRL_SIG_reg[0][5]'
INFO: [Synth 8-

INFO: [Synth 8-3333] propagating constant 0 across sequential element (dense_array_array_ap_fixed_16_6_5_3_0_1u_config8_U0/grp_dense_wrapper_ap_fixed_ap_fixed_16_6_5_3_0_config8_s_fu_2095/\trunc_ln708_57_reg_54985_pp0_iter1_reg_reg[0] )
INFO: [Synth 8-3333] propagating constant 0 across sequential element (dense_array_array_ap_fixed_16_6_5_3_0_1u_config8_U0/grp_dense_wrapper_ap_fixed_ap_fixed_16_6_5_3_0_config8_s_fu_2095/\trunc_ln708_57_reg_54985_pp0_iter1_reg_reg[1] )
INFO: [Synth 8-3333] propagating constant 0 across sequential element (dense_array_array_ap_fixed_16_6_5_3_0_1u_config8_U0/grp_dense_wrapper_ap_fixed_ap_fixed_16_6_5_3_0_config8_s_fu_2095/\trunc_ln708_57_reg_54985_pp0_iter1_reg_reg[2] )
INFO: [Synth 8-3333] propagating constant 0 across sequential element (dense_array_array_ap_fixed_16_6_5_3_0_1u_config8_U0/grp_dense_wrapper_ap_fixed_ap_fixed_16_6_5_3_0_config8_s_fu_2095/\trunc_ln708_57_reg_54985_pp0_iter1_reg_reg[3] )
INFO: [Synth 8-3333] propagating constant 0 across s


Report RTL Partitions: 
+------+---------------+------------+----------+
|      |RTL Partition  |Replication |Instances |
+------+---------------+------------+----------+
|1     |myproject__GB0 |           1|     23872|
|2     |myproject__GB1 |           1|     14981|
+------+---------------+------------+----------+
No constraint files found.
---------------------------------------------------------------------------------
Start Timing Optimization
---------------------------------------------------------------------------------
---------------------------------------------------------------------------------
Finished Timing Optimization : Time (s): cpu = 00:00:47 ; elapsed = 00:00:48 . Memory (MB): peak = 2194.309 ; gain = 653.016 ; free physical = 23559 ; free virtual = 26371
---------------------------------------------------------------------------------

Report RTL Partitions: 
+------+---------------+------------+----------+
|      |RTL Partition  |Replication |Instances |
+----

INFO: [Project 1-571] Translating synthesized netlist
Netlist sorting complete. Time (s): cpu = 00:00:00.12 ; elapsed = 00:00:00.12 . Memory (MB): peak = 2204.637 ; gain = 0.000 ; free physical = 23648 ; free virtual = 26462
INFO: [Netlist 29-17] Analyzing 1341 Unisim elements for replacement
INFO: [Netlist 29-28] Unisim Transformation completed in 0 CPU seconds
INFO: [Project 1-570] Preparing netlist for logic optimization
INFO: [Opt 31-138] Pushed 0 inverter(s) to 0 load pin(s).
Netlist sorting complete. Time (s): cpu = 00:00:00 ; elapsed = 00:00:00 . Memory (MB): peak = 2252.484 ; gain = 0.000 ; free physical = 23600 ; free virtual = 26414
INFO: [Project 1-111] Unisim Transformation Summary:
No Unisim elements were transformed.

INFO: [Common 17-83] Releasing license: Synthesis
455 Infos, 57 Warnings, 0 Critical Warnings and 0 Errors encountered.
synth_design completed successfully
synth_design: Time (s): cpu = 00:01:03 ; elapsed = 00:01:19 . Memory (MB): peak = 2252.484 ; gain = 73

{'EstimatedClockPeriod': '5.065',
 'BestLatency': '66',
 'WorstLatency': '66',
 'IntervalMin': '45',
 'IntervalMax': '45',
 'BRAM_18K': '0',
 'DSP48E': '55',
 'FF': '18994',
 'LUT': '29630',
 'URAM': '0',
 'AvailableBRAM_18K': '280',
 'AvailableDSP48E': '220',
 'AvailableFF': '106400',
 'AvailableLUT': '53200',
 'AvailableURAM': '0'}

In [9]:
hls4ml.report.read_vivado_report('quantized_pruned_cnn/myproject_prj/solution1')

Unable to read project data. Exiting.
